<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/cashflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for formatting
def format_rate(r):
    return f"{r * 100:.4f}%"

def format_vol(v):
    return f"{v * 100:.4f}%"

# Helper class similar to C++ CommonVars struct
class CommonVars:
    def __init__(self):
        self.nominals = [100.0] # Use list for Python bindings
        self.frequency = ql.Annual # Changed for this test suite
        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        # Changed to Euribor1Y for Annual frequency
        self.index = ql.Euribor1Y(self.termStructure)
        self.calendar = self.index.fixingCalendar()
        self.convention = ql.ModifiedFollowing
        # Use a fixed date for reproducibility
        self.today = ql.Date(15, ql.May, 2024) # Example fixed date
        # Set evaluation date *globally* for calculations relying on it
        # ql.Settings.instance().evaluationDate = self.today # Done in test setUp
        self.settlementDays = 2
        self.fixingDays = 2
        self.settlement = self.calendar.advance(self.today, self.settlementDays, ql.Days)
        # Link term structure AFTER settlement date is defined
        self.termStructure.linkTo(ql.FlatForward(self.settlement, 0.05,
                                                 ql.ActualActual(ql.ActualActual.ISDA)))
        # Other common variables from C++ test suite (potentially unused in translated tests)
        self.length = 20
        self.volatility = 0.20
        self.caps = []
        self.floors = []


# Test Suite
class CashFlowTests(unittest.TestCase):

    def setUp(self):
        """Set up the global evaluation date before each test."""
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Use a fixed date for consistent test runs
        self.today = ql.Date(15, ql.May, 2024)
        ql.Settings.instance().evaluationDate = self.today

    def tearDown(self):
        """Restore the global evaluation date after each test."""
        ql.Settings.instance().evaluationDate = self.saved_eval_date
        # Clean up potentially modified global settings
        ql.Settings.instance().includeReferenceDateEvents = True # Restore default? Check QL default. Default is true.
        ql.Settings.instance().includeTodaysCashFlows = None # Restore default (nullopt)

    def testSettings(self):
        """Testing cash-flow settings."""
        print("Testing cash-flow settings...")

        today = ql.Settings.instance().evaluationDate

        # cash flows at T+0, T+1, T+2
        leg = []
        for i in range(3):
            leg.append(ql.SimpleCashFlow(1.0, today + i)) # Uses ql.Days implicitly

        def check_inclusion(cf_index, days_offset, expected_outcome, settings_desc):
            # hasOccurred(ref_date) returns True if the payment has already happened
            # We want to check if it's *included* in NPV calculation based on ref_date
            # hasOccurred(ref_date) means it's typically *excluded* unless includeRef=True
            # The C++ test checks `(!leg[n]->hasOccurred(today+days)) != expected`
            # which simplifies to `leg[n]->hasOccurred(today+days) == expected` if expected means "has occurred"
            # Let's redefine `expected` to mean "is included" for clarity.
            # hasOccurred(refDate): true if d <= refDate
            # NPV includes if d > refDate (standard) OR d == refDate and includeRefDate is true.
            # C++ test's `expected`: True if included, False if excluded.
            # So, we check `not cf.hasOccurred(ref_date, includeRef)` matches expected? No, NPV logic is complex.
            # Let's stick to the C++ test's logic: `not cf.hasOccurred(ref_date)` should match `expected` value.

            ref_date = today + days_offset
            # Use the global settings for includeReferenceDateEvents and includeTodaysCashFlows
            # hasOccurred signature: bool hasOccurred(const Date& refDate = Date(), ext::optional<bool> includeRefDate = ext::nullopt) const;

            # Mimic C++ logic which implicitly uses settings
            # Python `hasOccurred` directly takes includeRefDate override.
            # To test settings, we must rely on functions that use them, like CashFlows.npv or implicit behavior.
            # Let's stick to the C++ test's direct check on hasOccurred logic.
            # hasOccurred(ref) returns true if cf.date() <= ref
            # We need to carefully map the C++ test logic.
            # CHECK_INCLUSION(n, days, expected) means:
            # If expected is TRUE, the condition (!leg[n]->hasOccurred(today+days)) should be TRUE.
            # If expected is FALSE, the condition (!leg[n]->hasOccurred(today+days)) should be FALSE.
            # So, `expected` directly maps to `not leg[n]->hasOccurred(today+days)`

            # The optional<bool> includeTodays param needs careful mapping.
            # QL Python's hasOccurred(refDate, includeRefDate) allows overriding the setting.
            # To test the *setting*, we should perhaps not pass the second arg and rely on default behavior?
            # Default is includeRefDate = None, which means use Settings.

            calculated_not_occurred = not leg[cf_index].hasOccurred(ref_date) # Relies on Settings via None default

            self.assertEqual(calculated_not_occurred, expected_outcome,
                             msg=f"{settings_desc}: cashflow at T+{cf_index} "
                                 f"{'should be included' if expected_outcome else 'should NOT be included'} "
                                 f"at ref_date T+{days_offset}. "
                                 f"hasOccurred({ref_date})={leg[cf_index].hasOccurred(ref_date)}")


        # === Case 1 ===
        ql.Settings.instance().includeReferenceDateEvents = False
        ql.Settings.instance().includeTodaysCashFlows = None
        desc1 = "Case 1 (inclRef=F, today=None)"
        check_inclusion(0, 0, False, desc1) # T+0 cf vs T+0 ref -> Occurred=T (<=), NotOccurred=F -> Excluded
        check_inclusion(0, 1, False, desc1) # T+0 cf vs T+1 ref -> Occurred=T (<=), NotOccurred=F -> Excluded
        check_inclusion(1, 0, True,  desc1) # T+1 cf vs T+0 ref -> Occurred=F (>),  NotOccurred=T -> Included
        check_inclusion(1, 1, False, desc1) # T+1 cf vs T+1 ref -> Occurred=T (<=), NotOccurred=F -> Excluded
        check_inclusion(1, 2, False, desc1) # T+1 cf vs T+2 ref -> Occurred=T (<=), NotOccurred=F -> Excluded
        check_inclusion(2, 1, True,  desc1) # T+2 cf vs T+1 ref -> Occurred=F (>),  NotOccurred=T -> Included
        check_inclusion(2, 2, False, desc1) # T+2 cf vs T+2 ref -> Occurred=T (<=), NotOccurred=F -> Excluded
        check_inclusion(2, 3, False, desc1) # T+2 cf vs T+3 ref -> Occurred=T (<=), NotOccurred=F -> Excluded

        # === Case 2 ===
        ql.Settings.instance().includeReferenceDateEvents = False
        ql.Settings.instance().includeTodaysCashFlows = False # Explicit override for today
        desc2 = "Case 2 (inclRef=F, today=F)"
        # Note: includeTodaysCashFlows affects *only* when refDate == Settings::evaluationDate()
        # So, check_inclusion needs to be aware of the global evaluation date.
        # Our check_inclusion uses `ref_date = today + days_offset`.
        # The override only matters when `days_offset == 0`.
        check_inclusion(0, 0, False, desc2) # T+0 cf vs T+0 ref (eval date). Occurred=T. Override applies -> Excluded
        check_inclusion(0, 1, False, desc2) # T+0 cf vs T+1 ref. Occurred=T -> Excluded (override doesn't apply)
        check_inclusion(1, 0, True,  desc2) # T+1 cf vs T+0 ref. Occurred=F -> Included
        check_inclusion(1, 1, False, desc2) # T+1 cf vs T+1 ref. Occurred=T -> Excluded
        check_inclusion(1, 2, False, desc2) # T+1 cf vs T+2 ref. Occurred=T -> Excluded
        check_inclusion(2, 1, True,  desc2) # T+2 cf vs T+1 ref. Occurred=F -> Included
        check_inclusion(2, 2, False, desc2) # T+2 cf vs T+2 ref. Occurred=T -> Excluded
        check_inclusion(2, 3, False, desc2) # T+2 cf vs T+3 ref. Occurred=T -> Excluded

        # === Case 3 ===
        ql.Settings.instance().includeReferenceDateEvents = True
        ql.Settings.instance().includeTodaysCashFlows = None
        desc3 = "Case 3 (inclRef=T, today=None)"
        check_inclusion(0, 0, True,  desc3) # T+0 cf vs T+0 ref. Occurred=T (<=). inclRef=T -> Included
        check_inclusion(0, 1, False, desc3) # T+0 cf vs T+1 ref. Occurred=T (<=). -> Excluded
        check_inclusion(1, 0, True,  desc3) # T+1 cf vs T+0 ref. Occurred=F (>). -> Included
        check_inclusion(1, 1, True,  desc3) # T+1 cf vs T+1 ref. Occurred=T (<=). inclRef=T -> Included
        check_inclusion(1, 2, False, desc3) # T+1 cf vs T+2 ref. Occurred=T (<=). -> Excluded
        check_inclusion(2, 1, True,  desc3) # T+2 cf vs T+1 ref. Occurred=F (>). -> Included
        check_inclusion(2, 2, True,  desc3) # T+2 cf vs T+2 ref. Occurred=T (<=). inclRef=T -> Included
        check_inclusion(2, 3, False, desc3) # T+2 cf vs T+3 ref. Occurred=T (<=). -> Excluded

        # === Case 4 ===
        ql.Settings.instance().includeReferenceDateEvents = True
        ql.Settings.instance().includeTodaysCashFlows = True # Explicit override for today
        desc4 = "Case 4 (inclRef=T, today=T)"
        check_inclusion(0, 0, True,  desc4) # T+0 cf vs T+0 ref (eval date). Occurred=T. Override applies -> Included
        check_inclusion(0, 1, False, desc4) # T+0 cf vs T+1 ref. Occurred=T -> Excluded (override doesn't apply)
        check_inclusion(1, 0, True,  desc4) # T+1 cf vs T+0 ref. Occurred=F -> Included
        check_inclusion(1, 1, True,  desc4) # T+1 cf vs T+1 ref. Occurred=T. inclRef=T -> Included
        check_inclusion(1, 2, False, desc4) # T+1 cf vs T+2 ref. Occurred=T -> Excluded
        check_inclusion(2, 1, True,  desc4) # T+2 cf vs T+1 ref. Occurred=F -> Included
        check_inclusion(2, 2, True,  desc4) # T+2 cf vs T+2 ref. Occurred=T. inclRef=T -> Included
        check_inclusion(2, 3, False, desc4) # T+2 cf vs T+3 ref. Occurred=T -> Excluded

        # === Case 5 ===
        ql.Settings.instance().includeReferenceDateEvents = True
        ql.Settings.instance().includeTodaysCashFlows = False # Explicit override for today
        desc5 = "Case 5 (inclRef=T, today=F)"
        check_inclusion(0, 0, False, desc5) # T+0 cf vs T+0 ref (eval date). Occurred=T. Override applies -> Excluded
        check_inclusion(0, 1, False, desc5) # T+0 cf vs T+1 ref. Occurred=T -> Excluded
        check_inclusion(1, 0, True,  desc5) # T+1 cf vs T+0 ref. Occurred=F -> Included
        check_inclusion(1, 1, True,  desc5) # T+1 cf vs T+1 ref. Occurred=T. inclRef=T -> Included
        check_inclusion(1, 2, False, desc5) # T+1 cf vs T+2 ref. Occurred=T -> Excluded
        check_inclusion(2, 1, True,  desc5) # T+2 cf vs T+1 ref. Occurred=F -> Included
        check_inclusion(2, 2, True,  desc5) # T+2 cf vs T+2 ref. Occurred=T. inclRef=T -> Included
        check_inclusion(2, 3, False, desc5) # T+2 cf vs T+3 ref. Occurred=T -> Excluded

        # --- Test NPV calculation ---
        # Zero discount rate for simplicity
        no_discount = ql.InterestRate(0.0, ql.Actual365Fixed(), ql.Continuous, ql.Annual)

        def check_npv(includeRefDateEventsInNPV, expected_npv, settings_desc):
            # CashFlows.npv signature:
            # npv(leg, YieldTermStructure discountCurve, bool includeSettlementDateFlows, Date settlementDate = Date(), Date npvDate = Date())
            # npv(leg, InterestRate y, bool includeSettlementDateFlows, Date settlementDate = Date(), Date npvDate = Date())
            # npv(leg, YieldTermStructure discountCurve, Date settlementDate = Date(), Date npvDate = Date())

            # The C++ CHECK_NPV macro uses `CashFlows::npv(leg, no_discount, includeRef, today)`
            # This corresponds to the second signature. `includeRef` controls behavior for flows on `today`.
            # QL Python CashFlows.npv doesn't seem to directly expose the override for today's cashflow setting *via* the `includeRef` bool.
            # The `includeSettlementDateFlows` flag in the Python signatures relates to the *settlement date* passed, not the *evaluation date* override.
            # The C++ test seems to imply the `includeRef` bool passed to `npv` *does* consider the `includeTodaysCashFlows` setting.
            # Let's test this assumption in Python.

            # We use the version taking InterestRate.
            # settlementDate defaults to Settings().evaluationDate, npvDate defaults to settlementDate.
            # We pass `today` explicitly for `npvDate` to match C++? No, C++ passes `today` as `npvDate` implicitly.
            # Let's use the default settlement/npv date behavior.
            # The bool flag controls whether flows on the *settlement date* are included.
            # The C++ test passes `today` as the `npvDate` implicitly via the overload used.
            # The `includeRef` bool seems to map to `includeSettlementDateFlows`.
            # We need to test how this interacts with the `Settings`.

            # Let's try calling npv with explicit settlement/npv date = today
            npv_calc = ql.CashFlows.npv(leg, no_discount, includeRefDateEventsInNPV, today, today)

            self.assertAlmostEqual(npv_calc, expected_npv, delta=1e-6,
                                   msg=(f"{settings_desc}: NPV mismatch (includeRef={includeRefDateEventsInNPV}):\n"
                                        f"    calculated: {npv_calc}\n"
                                        f"    expected: {expected_npv}"))

        # Test NPV with different settings

        # Setting: includeRef=T, today=None (like Case 3)
        ql.Settings.instance().includeReferenceDateEvents = True
        ql.Settings.instance().includeTodaysCashFlows = None
        desc_npv1 = "NPV (inclRef=T, today=None)"
        # includeRef=F passed to npv: Excludes flows on settlement date (today). NPV = 1(T+1) + 1(T+2) = 2.0
        check_npv(False, 2.0, desc_npv1)
        # includeRef=T passed to npv: Includes flows on settlement date (today). NPV = 1(T+0) + 1(T+1) + 1(T+2) = 3.0
        check_npv(True, 3.0, desc_npv1)

        # Setting: includeRef=T, today=F (like Case 5)
        ql.Settings.instance().includeReferenceDateEvents = True
        ql.Settings.instance().includeTodaysCashFlows = False
        desc_npv2 = "NPV (inclRef=T, today=F)"
        # includeRef=F passed to npv: Excludes flows on settlement date (today). NPV = 1(T+1) + 1(T+2) = 2.0
        check_npv(False, 2.0, desc_npv2)
        # includeRef=T passed to npv: Tries to include flows on settlement date (today).
        # *BUT* today is the eval date, and includeTodaysCashFlows=False overrides. NPV = 1(T+1) + 1(T+2) = 2.0
        check_npv(True, 2.0, desc_npv2)

    def testAccessViolation(self):
        """Testing dynamic cast of coupon in Black pricer."""
        print("Testing dynamic cast of coupon in Black pricer...")
        original_eval_date = ql.Settings.instance().evaluationDate

        todaysDate = ql.Date(7, ql.April, 2010)
        settlementDate = ql.Date(9, ql.April, 2010)
        ql.Settings.instance().evaluationDate = todaysDate
        calendar = ql.TARGET()

        rhTermStructure = ql.YieldTermStructureHandle(
            ql.FlatForward(settlementDate, 0.04875825, ql.Actual365Fixed()))

        volatility = 0.10
        vol_handle = ql.OptionletVolatilityStructureHandle(
            ql.ConstantOptionletVolatility(2, calendar, ql.ModifiedFollowing,
                                           volatility, ql.Actual365Fixed()))

        index3m = ql.USDLibor(ql.Period(3, ql.Months), rhTermStructure)

        payDate = ql.Date(20, ql.December, 2013)
        startDate = ql.Date(20, ql.September, 2013)
        endDate = ql.Date(20, ql.December, 2013)
        spread = 0.0115 # C++ has spread / 100, assuming spread is in %, but variable name suggests rate. Let's assume rate.

        # Create coupon
        coupon = ql.FloatingRateCoupon(payDate, 100.0, startDate, endDate,
                                       index3m.fixingDays(), index3m, 1.0, spread)

        # Create and set pricer
        pricer = ql.BlackIborCouponPricer(vol_handle)
        coupon.setPricer(pricer)

        # The test is to see if calculating amount causes issues
        try:
            # Accessing amount might trigger calculations involving the pricer
            _ = coupon.amount()
            # If no exception, it passes the original C++ test's intent
            # (where it might have crashed before)
        except ql.Error as e:
            # QL Errors are expected if e.g., fixing is missing.
            # The C++ test expects either success or a QL Error, not a crash.
            # So, catching ql.Error means it behaves correctly (doesn't crash).
            print(f"Caught expected QL error: {e}")
            pass
        except Exception as e:
            # Catch any other unexpected exception
            self.fail(f"Unexpected exception during coupon.amount(): {e}")

        ql.Settings.instance().evaluationDate = original_eval_date

    def testDefaultSettlementDate(self):
        """Testing default evaluation date in cashflows methods."""
        print("Testing default evaluation date in cashflows methods...")
        # today is already set in setUp
        today = ql.Settings.instance().evaluationDate

        schedule = ql.MakeSchedule(today - ql.Period(2, ql.Months), today + ql.Period(4, ql.Months)) \
                     .withFrequency(ql.Semiannual) \
                     .withCalendar(ql.TARGET()) \
                     .withConvention(ql.Unadjusted) \
                     .backwards().schedule()

        leg = ql.FixedRateLeg(schedule) \
                .withNotionals(100.0) \
                .withCouponRates(0.03, ql.Actual360()) \
                .withPaymentCalendar(ql.TARGET()) \
                .withPaymentAdjustment(ql.Following)

        # Test functions using default settlement (evaluation date)
        # includeRefDate = False means exclude flows ON the settlement date
        accruedPeriod = ql.CashFlows.accruedPeriod(leg, False)
        self.assertNotEqual(accruedPeriod, 0.0, "null accrued period with default settlement date")

        accruedDays = ql.CashFlows.accruedDays(leg, False)
        self.assertNotEqual(accruedDays, 0, "no accrued days with default settlement date")

        accruedAmount = ql.CashFlows.accruedAmount(leg, False)
        # This might legitimately be 0.0 if eval date is exactly start date of period
        # Let's check if the period is non-zero first
        if accruedPeriod > 1e-9:
            self.assertNotAlmostEqual(accruedAmount, 0.0, delta=1e-9, msg="potentially incorrect null accrued amount with default settlement date")
        else:
            print("Accrued period is zero, accrued amount test skipped.") # Or assert it IS zero

    # testNullFixingDays requires usingAtParCoupons() precondition,
    # which depends on a specific build flag or setting not easily checkable here.
    # We'll run it assuming the setting is compatible or the test logic itself is robust.
    @unittest.skipIf(not ql.IborCoupon.Settings.instance().usingAtParCoupons(), "Skipping testNullFixingDays: requires usingAtParCoupons setting")
    def testNullFixingDays(self):
        """Testing ibor leg construction with null fixing days."""
        print("Testing ibor leg construction with null fixing days...")
        today = ql.Settings.instance().evaluationDate

        schedule = ql.MakeSchedule(today - ql.Period(2, ql.Months), today + ql.Period(4, ql.Months)) \
                     .withFrequency(ql.Semiannual) \
                     .withCalendar(ql.TARGET()) \
                     .withConvention(ql.Following) \
                     .backwards().schedule()

        index = ql.USDLibor(ql.Period(6, ql.Months))

        try:
            # Pass an empty list [] for fixing days to simulate Null<Natural>
            leg = ql.IborLeg(schedule, index) \
                    .withNotionals(100.0) \
                    .withFixingDays([]) # Pass empty list for fixing days
            # Check if leg construction succeeded (no throw)
            self.assertTrue(len(leg) > 0, "Leg construction with null fixing days failed")
        except Exception as e:
            self.fail(f"IborLeg construction with null fixing days threw an exception: {e}")

    def testExCouponDates(self):
        """Testing ex-coupon date calculation."""
        print("Testing ex-coupon date calculation...")
        today = ql.Date(15, ql.May, 2024) # Use a fixed date
        # ql.Settings.instance().evaluationDate = today # Already set in setUp

        schedule = ql.MakeSchedule(today, today + ql.Period(5, ql.Years)) \
                     .withFrequency(ql.Monthly) \
                     .withCalendar(ql.TARGET()) \
                     .withConvention(ql.Following).schedule()

        # No ex-coupon
        leg1 = ql.FixedRateLeg(schedule).withNotionals(100.0).withCouponRates(0.03, ql.Actual360())
        for cf in leg1:
            coupon = ql.as_coupon(cf)
            if coupon:
                self.assertEqual(coupon.exCouponDate(), ql.Date(),
                                 f"ex-coupon date found ({coupon.exCouponDate()}) for coupon {coupon.date()}, none expected")

        index = ql.Euribor3M()
        leg2 = ql.IborLeg(schedule, index).withNotionals(100.0)
        for cf in leg2:
            coupon = ql.as_coupon(cf)
            if coupon:
                self.assertEqual(coupon.exCouponDate(), ql.Date(),
                                 f"ex-coupon date found ({coupon.exCouponDate()}) for float coupon {coupon.date()}, none expected")

        # Calendar days
        ex_period_cal = ql.Period(2, ql.Days)
        ex_cal_cal = ql.NullCalendar()
        ex_conv_cal = ql.Unadjusted
        leg5 = ql.FixedRateLeg(schedule).withNotionals(100.0).withCouponRates(0.03, ql.Actual360()) \
                 .withExCouponPeriod(ex_period_cal, ex_cal_cal, ex_conv_cal, False)
        for cf in leg5:
            coupon = ql.as_coupon(cf)
            if coupon:
                expected_ex_date = coupon.accrualEndDate() - ex_period_cal # Subtract period directly
                self.assertEqual(coupon.exCouponDate(), expected_ex_date,
                                 f"ex-coupon (cal days): got {coupon.exCouponDate()}, expected {expected_ex_date} for coupon {coupon.date()}")

        leg6 = ql.IborLeg(schedule, index).withNotionals(100.0) \
                 .withExCouponPeriod(ex_period_cal, ex_cal_cal, ex_conv_cal, False)
        for cf in leg6:
            coupon = ql.as_coupon(cf)
            if coupon:
                expected_ex_date = coupon.accrualEndDate() - ex_period_cal
                self.assertEqual(coupon.exCouponDate(), expected_ex_date,
                                 f"ex-coupon float (cal days): got {coupon.exCouponDate()}, expected {expected_ex_date} for coupon {coupon.date()}")

        # Business days
        ex_period_bday = ql.Period(2, ql.Days)
        ex_cal_bday = ql.TARGET()
        ex_conv_bday = ql.Preceding
        leg7 = ql.FixedRateLeg(schedule).withNotionals(100.0).withCouponRates(0.03, ql.Actual360()) \
                 .withExCouponPeriod(ex_period_bday, ex_cal_bday, ex_conv_bday, False)
        for cf in leg7:
            coupon = ql.as_coupon(cf)
            if coupon:
                expected_ex_date = ex_cal_bday.advance(coupon.accrualEndDate(), -ex_period_bday, ex_conv_bday)
                self.assertEqual(coupon.exCouponDate(), expected_ex_date,
                                 f"ex-coupon (biz days): got {coupon.exCouponDate()}, expected {expected_ex_date} for coupon {coupon.date()}")

        leg8 = ql.IborLeg(schedule, index).withNotionals(100.0) \
                 .withExCouponPeriod(ex_period_bday, ex_cal_bday, ex_conv_bday, False)
        for cf in leg8:
             coupon = ql.as_coupon(cf)
             if coupon:
                 expected_ex_date = ex_cal_bday.advance(coupon.accrualEndDate(), -ex_period_bday, ex_conv_bday)
                 self.assertEqual(coupon.exCouponDate(), expected_ex_date,
                                  f"ex-coupon float (biz days): got {coupon.exCouponDate()}, expected {expected_ex_date} for coupon {coupon.date()}")


    def testIrregularFirstCouponReferenceDatesAtEndOfMonth(self):
        """Testing irregular first coupon reference dates with end of month enabled."""
        print("Testing irregular first coupon reference dates with end of month enabled...")
        schedule = ql.MakeSchedule(ql.Date(17, ql.January, 2017), ql.Date(28, ql.February, 2018)) \
                     .withFrequency(ql.Semiannual) \
                     .withConvention(ql.Unadjusted) \
                     .endOfMonth(True) \
                     .backwards().schedule()

        leg = ql.FixedRateLeg(schedule) \
                .withNotionals(100.0) \
                .withCouponRates(0.01, ql.Actual360())

        firstCoupon = ql.as_coupon(leg[0])
        self.assertIsNotNone(firstCoupon, "First element is not a Coupon")

        # Expected reference start: Semiannual backwards from Feb 28, 2018 EOM is Aug 31, 2017.
        # Previous is Feb 28, 2017 EOM.
        # C++ expects Aug 31, 2016. This seems wrong based on schedule dates. Let's re-verify schedule.
        # Dates: [31-Jan-2017, 31-Aug-2017, 28-Feb-2018]. First coupon period: Jan 17, 2017 to Aug 31, 2017.
        # Reference period start should align with theoretical previous coupon end date if regular.
        # Previous coupon would end Aug 31, 2016. C++ expected seems correct.
        self.assertEqual(firstCoupon.referencePeriodStart(), ql.Date(31, ql.August, 2016),
                         f"Expected reference start date at end of month, got {firstCoupon.referencePeriodStart()}")


    def testIrregularFirstCouponReferenceDatesAtEndOfCalendarMonth(self):
         """Testing irregular first coupon reference dates at end of calendar month with end of month enabled."""
         print("Testing irregular first coupon reference dates at end of calendar month with end of month enabled...")
         schedule = ql.MakeSchedule(ql.Date(30, ql.September, 2017), ql.Date(30, ql.September, 2022)) \
                      .withTenor(ql.Period(6, ql.Months)) \
                      .withCalendar(ql.UnitedStates(ql.UnitedStates.GovernmentBond)) \
                      .withConvention(ql.Unadjusted) \
                      .withTerminationDateConvention(ql.Unadjusted) \
                      .withFirstDate(ql.Date(31, ql.March, 2018)) \
                      .withNextToLastDate(ql.Date(31, ql.March, 2022)) \
                      .endOfMonth(True) \
                      .backwards().schedule()

         leg = ql.FixedRateLeg(schedule) \
                 .withNotionals(100.0) \
                 .withCouponRates(0.01875, ql.ActualActual(ql.ActualActual.ISMA))

         firstCoupon = ql.as_coupon(leg[0])
         self.assertIsNotNone(firstCoupon)
         # First coupon period: Sep 30, 2017 to Mar 31, 2018.
         # Theoretical previous period end: Mar 31, 2017 (EOM adjusted).
         # C++ test expects Sep 30, 2017. Let's check.
         self.assertEqual(firstCoupon.referencePeriodStart(), ql.Date(30, ql.September, 2017))

         # Check amount
         # Period Sep 30, 2017 to Mar 31, 2018. Day count ISMA.
         # ql.ActualActual(ql.ActualActual.ISMA).yearFraction(ql.Date(30,9,2017), ql.Date(31,3,2018)) = 0.5
         # Expected amount = 100.0 * 0.01875 * 0.5 = 0.9375
         self.assertAlmostEqual(firstCoupon.amount(), 0.9375, delta=0.0001)

    def testIrregularLastCouponReferenceDatesAtEndOfMonth(self):
        """Testing irregular last coupon reference dates with end of month enabled."""
        print("Testing irregular last coupon reference dates with end of month enabled...")
        schedule = ql.MakeSchedule(ql.Date(17, ql.January, 2017), ql.Date(15, ql.September, 2018)) \
                     .withNextToLastDate(ql.Date(28, ql.February, 2018)) \
                     .withFrequency(ql.Semiannual) \
                     .withConvention(ql.Unadjusted) \
                     .endOfMonth(True) \
                     .backwards().schedule()

        leg = ql.FixedRateLeg(schedule) \
                .withNotionals(100.0) \
                .withCouponRates(0.01, ql.Actual360())

        lastCoupon = ql.as_coupon(leg[-1])
        self.assertIsNotNone(lastCoupon)
        # Last coupon period: Feb 28, 2018 to Sep 15, 2018.
        # Theoretical next period start: Aug 31, 2018 (EOM adjusted).
        self.assertEqual(lastCoupon.referencePeriodEnd(), ql.Date(31, ql.August, 2018),
                         f"Expected reference end date at end of month, got {lastCoupon.referencePeriodEnd()}")


    def testPartialScheduleLegConstruction(self):
         """Testing leg construction with partial schedule."""
         print("Testing leg construction with partial schedule...")
         # Schedule with irregular first and last periods
         schedule1 = ql.MakeSchedule(ql.Date(15, ql.September, 2017), ql.Date(30, ql.September, 2020)) \
                      .withNextToLastDate(ql.Date(25, ql.September, 2020)) \
                      .withFrequency(ql.Semiannual) \
                      .backwards().schedule()

         # Schedule from dates with metadata (check regularity)
         is_regular1 = [schedule1.isRegular(i) for i in range(1, len(schedule1))] # Check original schedule
         schedule2 = ql.Schedule(schedule1.dates(), ql.NullCalendar(), ql.Unadjusted, ql.Unadjusted,
                                 ql.Period(6, ql.Months), ql.DateGeneration.Backward, schedule1.endOfMonth(), is_regular1)

         # Schedule from dates without metadata
         schedule3 = ql.Schedule(schedule1.dates())

         # Fixed Legs
         leg1 = ql.FixedRateLeg(schedule1).withNotionals(100.0).withCouponRates(0.01, ql.ActualActual(ql.ActualActual.ISMA))
         leg2 = ql.FixedRateLeg(schedule2).withNotionals(100.0).withCouponRates(0.01, ql.ActualActual(ql.ActualActual.ISMA))
         leg3 = ql.FixedRateLeg(schedule3).withNotionals(100.0).withCouponRates(0.01, ql.ActualActual(ql.ActualActual.ISMA))

         # Check reference periods
         firstCpn1 = ql.as_fixed_rate_coupon(leg1[0]); lastCpn1 = ql.as_fixed_rate_coupon(leg1[-1])
         self.assertEqual(firstCpn1.referencePeriodStart(), ql.Date(25, 3, 2017))
         self.assertEqual(firstCpn1.referencePeriodEnd(), ql.Date(25, 9, 2017))
         self.assertEqual(lastCpn1.referencePeriodStart(), ql.Date(25, 9, 2020))
         self.assertEqual(lastCpn1.referencePeriodEnd(), ql.Date(25, 3, 2021))

         firstCpn2 = ql.as_fixed_rate_coupon(leg2[0]); lastCpn2 = ql.as_fixed_rate_coupon(leg2[-1])
         self.assertEqual(firstCpn2.referencePeriodStart(), ql.Date(25, 3, 2017))
         self.assertEqual(firstCpn2.referencePeriodEnd(), ql.Date(25, 9, 2017))
         self.assertEqual(lastCpn2.referencePeriodStart(), ql.Date(25, 9, 2020))
         self.assertEqual(lastCpn2.referencePeriodEnd(), ql.Date(25, 3, 2021))

         firstCpn3 = ql.as_fixed_rate_coupon(leg3[0]); lastCpn3 = ql.as_fixed_rate_coupon(leg3[-1])
         self.assertEqual(firstCpn3.referencePeriodStart(), ql.Date(15, 9, 2017)) # No metadata, uses schedule dates
         self.assertEqual(firstCpn3.referencePeriodEnd(), ql.Date(25, 9, 2017)) # Uses schedule dates
         self.assertEqual(lastCpn3.referencePeriodStart(), ql.Date(25, 9, 2020)) # Uses schedule dates
         self.assertEqual(lastCpn3.referencePeriodEnd(), ql.Date(30, 9, 2020)) # Uses schedule dates

         # Floating Legs
         iborIndex = ql.USDLibor(ql.Period(3, ql.Months))
         legf1 = ql.IborLeg(schedule1, iborIndex).withNotionals(100.0).withPaymentDayCounter(ql.ActualActual(ql.ActualActual.ISMA))
         legf2 = ql.IborLeg(schedule2, iborIndex).withNotionals(100.0).withPaymentDayCounter(ql.ActualActual(ql.ActualActual.ISMA))
         legf3 = ql.IborLeg(schedule3, iborIndex).withNotionals(100.0).withPaymentDayCounter(ql.ActualActual(ql.ActualActual.ISMA))

         # Check reference periods for floating legs
         firstCpnF1 = ql.as_floating_rate_coupon(legf1[0]); lastCpnF1 = ql.as_floating_rate_coupon(legf1[-1])
         self.assertEqual(firstCpnF1.referencePeriodStart(), ql.Date(25, 3, 2017))
         self.assertEqual(firstCpnF1.referencePeriodEnd(), ql.Date(25, 9, 2017))
         self.assertEqual(lastCpnF1.referencePeriodStart(), ql.Date(25, 9, 2020))
         self.assertEqual(lastCpnF1.referencePeriodEnd(), ql.Date(25, 3, 2021))

         firstCpnF2 = ql.as_floating_rate_coupon(legf2[0]); lastCpnF2 = ql.as_floating_rate_coupon(legf2[-1])
         self.assertEqual(firstCpnF2.referencePeriodStart(), ql.Date(25, 3, 2017))
         self.assertEqual(firstCpnF2.referencePeriodEnd(), ql.Date(25, 9, 2017))
         self.assertEqual(lastCpnF2.referencePeriodStart(), ql.Date(25, 9, 2020))
         self.assertEqual(lastCpnF2.referencePeriodEnd(), ql.Date(25, 3, 2021))

         firstCpnF3 = ql.as_floating_rate_coupon(legf3[0]); lastCpnF3 = ql.as_floating_rate_coupon(legf3[-1])
         self.assertEqual(firstCpnF3.referencePeriodStart(), ql.Date(15, 9, 2017))
         self.assertEqual(firstCpnF3.referencePeriodEnd(), ql.Date(25, 9, 2017))
         self.assertEqual(lastCpnF3.referencePeriodStart(), ql.Date(25, 9, 2020))
         self.assertEqual(lastCpnF3.referencePeriodEnd(), ql.Date(30, 9, 2020))


    def testFixedIborCouponWithoutForecastCurve(self):
        """Testing past ibor coupon without forecast curve."""
        print("Testing past ibor coupon without forecast curve...")
        today = ql.Settings.instance().evaluationDate

        index = ql.USDLibor(ql.Period(6, ql.Months)) # No term structure needed if fixing is available
        calendar = index.fixingCalendar()

        fixingDate = calendar.advance(today, -2, ql.Months)
        pastFixing = 0.01
        index.addFixing(fixingDate, pastFixing)

        startDate = index.valueDate(fixingDate)
        endDate = index.maturityDate(startDate) # Maturity based on index tenor from value date

        coupon = ql.IborCoupon(endDate, 100.0, startDate, endDate, index.fixingDays(), index)

        # Create a dummy pricer (needed for amount calculation, even if vol=0)
        dummy_vol = ql.OptionletVolatilityStructureHandle(
            ql.ConstantOptionletVolatility(0, calendar, ql.Following, 0.0, index.dayCounter()))
        pricer = ql.BlackIborCouponPricer(dummy_vol)
        coupon.setPricer(pricer)

        try:
            amount = coupon.amount()
            # Check consistency
            expected = pastFixing * coupon.nominal() * coupon.accrualPeriod()
            self.assertAlmostEqual(amount, expected, delta=1e-8,
                                   msg=f"Amount mismatch: calc={amount}, exp={expected}")
        except Exception as e:
            self.fail(f"coupon.amount() raised an exception: {e}")

        # Clean up fixing to avoid interfering with other tests
        index.clearFixings()

    def testIborCouponKnowsWhenitHasFixed(self):
        """Testing that ibor coupon knows when it has fixed."""
        print("Testing that ibor coupon knows when it has fixed...")
        today = ql.Settings.instance().evaluationDate
        index = ql.Euribor3M()
        calendar = index.fixingCalendar()

        # Helper to create coupon for a fixing date
        def iborCouponForFixingDate(idx, fixing_dt):
             startDate = idx.valueDate(fixing_dt)
             endDate = idx.maturityDate(startDate) # Maturity based on start date
             coupon = ql.IborCoupon(endDate, 100.0, startDate, endDate, idx.fixingDays(), idx)
             # Add a dummy pricer if needed for methods like rate()
             dummy_vol = ql.OptionletVolatilityStructureHandle(
                 ql.ConstantOptionletVolatility(0, idx.fixingCalendar(), ql.Following, 0.0, idx.dayCounter()))
             pricer = ql.BlackIborCouponPricer(dummy_vol)
             coupon.setPricer(pricer)
             return coupon

        # Case 1: Fixing date in the past, no fixing provided
        fixing_past = calendar.advance(today, -1, ql.Days)
        coupon_past = iborCouponForFixingDate(index, fixing_past)
        index.clearFixings()
        # Coupon should know fixing date is in the past
        self.assertTrue(coupon_past.fixingDate() < today)
        # hasFixed() is expected True because the date is in the past
        self.assertTrue(coupon_past.hasFixed(), f"Coupon with fixing {fixing_past} should have hasFixed()=True")
        # Accessing rate() should fail if fixing is missing
        with self.assertRaises(Exception, msg="Accessing rate() for past fixing date without data should fail"):
             _ = coupon_past.rate()
        index.clearFixings() # Clean up

        # Case 2: Fixing date is today, enforce=False, no fixing
        coupon_today = iborCouponForFixingDate(index, today)
        ql.Settings.instance().enforcesTodaysHistoricFixings = False
        index.clearFixings()
        self.assertFalse(coupon_today.hasFixed(), f"Coupon with fixing {today} (enforce=F) should have hasFixed()=False")
        index.clearFixings()

        # Case 3: Fixing date is today, enforce=False, fixing available
        coupon_today_fixed = iborCouponForFixingDate(index, today)
        ql.Settings.instance().enforcesTodaysHistoricFixings = False
        index.addFixing(coupon_today_fixed.fixingDate(), 0.01)
        self.assertTrue(coupon_today_fixed.hasFixed(), f"Coupon with fixing {today} (enforce=F, fixed) should have hasFixed()=True")
        index.clearFixings()

        # Case 4: Fixing date is today, enforce=True, no fixing
        coupon_today_enforced = iborCouponForFixingDate(index, today)
        ql.Settings.instance().enforcesTodaysHistoricFixings = True
        index.clearFixings()
        # hasFixed() should be True because enforcement implies it *should* be fixed
        self.assertTrue(coupon_today_enforced.hasFixed(), f"Coupon with fixing {today} (enforce=T) should have hasFixed()=True")
        with self.assertRaises(Exception, msg="Accessing rate() for today fixing date with enforcement but no data should fail"):
            _ = coupon_today_enforced.rate()
        index.clearFixings()

        # Case 5: Fixing date in the future
        fixing_future = calendar.advance(today, 1, ql.Days)
        coupon_future = iborCouponForFixingDate(index, fixing_future)
        self.assertFalse(coupon_future.hasFixed(), f"Coupon with fixing {fixing_future} should have hasFixed()=False")

        # Restore default setting
        ql.Settings.instance().enforcesTodaysHistoricFixings = True


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)